In [1]:
from visualizer import calibration
#TODO: import from waveorder
import visual 

from PyQt5 import QtCore

from skimage.io import imread
from skimage.io.collection import alphanumeric_key
from dask import delayed
from glob import glob
import dask.array as da
import matplotlib.pyplot as plt
import numpy as np
import napari
from napari import Viewer
import os
import tifffile

from skimage.registration import phase_cross_correlation
import cv2 as cv
from skimage import io

import cupy as cp 
import imagej

%gui qt 
%matplotlib inline
plt.style.use('dark_background')


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [2]:
# main_folder =r'H:\Test\20220507_M25_ZW495_Egg_Timelapse'  #DIED 
# main_folder =r'H:\Test\20220507_M25_ZW495_19'   # 5 worm in all FOV and some movement
# main_folder =r'H:\Test\20220507_M25_ZW495_18'  #3 starts in FOV but then moves out
# main_folder =r'H:\Test\20220507_M25_ZW495_17'  # 3 some movement in 3D and some beads are on FOV
# main_folder =r'H:\Test\20220507_M25_ZW495_16'  #3 thicc one with barely any movement
# main_folder =r'H:\Test\20220507_M25_ZW495_15'  #3 just the tail
# main_folder =r'H:\Test\20220507_M25_ZW495_14' #5 2 worms in FOV.
# main_folder =r'H:\Test\20220507_M25_ZW495_13' #3 movement at the edge
# main_folder =r'H:\Test\20220507_M25_ZW495_12'  #3 not focus on center planes
# main_folder =r'H:\Test\20220507_M25_ZW495_11' #3 not much movement and out of focus on central planes
# main_folder =r'H:\Test\20220507_M25_ZW495_10'  #4 worth taking a look since they move in 3D
# main_folder =r'H:\Test\20220507_M25_ZW495_9' #5 moves in FOV. Curling up and then shooting out
# main_folder =r'H:\Test\20220507_M25_ZW495_8'  #3 movement out of FOV
# main_folder =r'H:\Test\20220507_M25_ZW495_7'  # 5 lots of worms in FOV coming in and out.
# main_folder =r'H:\Test\20220507_M25_ZW495_6'  # 5 lots of twitching in FOV
# main_folder =r'H:\Test\20220507_M25_ZW495_5'  #3 out FOV movement
# main_folder =r'H:\Test\20220507_M25_ZW495_4'  #5 amazing movement in 3D
# main_folder =r'H:\Test\20220507_M25_ZW495_3'  #5 movement in 3D
# main_folder =r'H:\Test\20220507_M25_ZW495_2'  #5 fast dynamics on second half
# main_folder =r'H:\Test\20220507_M25_ZW495_1'  #5 fast dynamics of big worm

# main_folder =r'H:\Data\KellerExplant_ian\20220508_M25_Iansample'  #5 fast dynamics of big worm

# main_folder =r'H:\Data\OH15265\20220508_M25_oh15265_1'  #3 first frames are slow
# main_folder =r'H:\Data\OH15265\20220508_M25_oh15265_1'   #5 fast dynamics of big worm
# main_folder =r'H:\Data\OH15265\20220508_M25_oh15265_1'   #5 fast dynamics of big worm
# main_folder =r'H:\Data\OH15265\20220508_M25_oh15265_1'  #5 fast dynamics of big worm
# main_folder =r'H:\Data\OH15265\20220508_M25_oh15265_1'   #5 fast dynamics of big worm
# main_folder =r'H:\Data\OH15265\20220508_M25_oh15265_1'   #5 fast dynamics of big worm

main_folder =r'H:\Data\KellerExplant_ian\20220508_M25_kelleexplant_ian'   #5 fast dynamics of big worm



#Scope Parameters
# FOV = 50e-6
cam_px = 6.0e-6
totalmag = 15.75
px_size_img = cam_px/totalmag
zstep = 2e-6
z_scale = zstep/px_size_img

print(px_size_img)
print(zstep/cam_px)
print(z_scale)

3.8095238095238096e-07
0.3333333333333333
5.25


### Loading center planes for evaluation

In [3]:
%%time
#Load Only center Camera to see if good FOV and signal

# center_planes = calibration.lazy_dask_stack(main_folder,num_cams=10,px_depth='uint8', height=608, width =808)

center_planes = calibration.lazy_dask_stack(main_folder,num_cams=5,px_depth='uint16', height=608, width =608)

CPU times: total: 93.8 ms
Wall time: 87.9 ms


In [4]:
napari.view_image(center_planes, name='center_planes',scale=[z_scale,1,1],multiscale=False)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 303.5, 303.5), zoom=0.8265625, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=2, last_used=0, range=((0.0, 60.0, 1.0), (0.0, 26.25, 5.25), (0.0, 608.0, 1.0), (0.0, 608.0, 1.0)), current_step=(30, 2, 304, 304), order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'center_planes' at 0x226f63b7dc0>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False, 

### Loading Dataset

In [5]:
%%time

# stack = calibration.lazy_dask_stack(main_folder,num_cams=25,px_depth='uint8', height=600, width =960)

stack = calibration.lazy_dask_stack(main_folder,num_cams=25,px_depth='uint16', height=608, width =608)

CPU times: total: 422 ms
Wall time: 434 ms


### Align Datasets with Known Offsets


In [6]:
# Load offsets from desired folder 
main_folder_offset = r'H:\Test\20220503_M25_1um_yg'
offsets_file_load = os.path.join(main_folder_offset,"offsets.csv")
with open(offsets_file_load) as file_name:
    offsets = np.loadtxt(file_name, delimiter=",")

In [12]:
from cupyx.scipy.ndimage import shift
import cupy as cp
# Generate our own shifted datset

offset_coordinates = offsets
with cp.cuda.Device(0):
    stack_aligned = []
    t,c,h,w = stack.shape
    shift_stack_cam = cp.zeros((c,h,w))
    columns = cp.zeros((offset_coordinates.shape[0],1))
    shift_stack_coord = cp.hstack((columns,offset_coordinates))
    print(shift_stack_coord.shape)
    print(stack.shape)
    for i in range(c):
        cp_stack = cp.array(stack[:,i,:,:])
        shift_stack_cam= shift(cp_stack,shift_stack_coord[i])
        stack_aligned.append(cp.asnumpy(shift_stack_cam))
        cp_stack = None
    stack_aligned = np.array(stack_aligned)
stack_aligned =  np.moveaxis(stack_aligned,0,1)

(25, 3)
(60, 25, 608, 608)


In [13]:
#Convert numpy to dask?
da_stack_align = da.from_array(stack_aligned)
da_stack_align[:,12,:,:]= da_stack_align[:,12,:,:]*1.3

print(da_stack_align)

dask.array<setitem, shape=(60, 25, 608, 608), dtype=uint16, chunksize=(60, 25, 152, 152), chunktype=numpy.ndarray>


In [8]:
viewer= napari.Viewer()

In [86]:
## View partial planes
da_stack_align_partial = da.from_array(stack_aligned[:,5:22,:,:])
viewer.add_image(da_stack_align_partial, name='dask_align',scale=[z_scale,1,1], multiscale=False)

<Image layer 'dask_align' at 0x22a595d8160>

In [87]:
viewer= napari.Viewer()

In [14]:
viewer.add_image(da_stack_align, name='dask_align',scale=[z_scale,1,1], multiscale=False)

<Image layer 'dask_align' at 0x226f62f11c0>

In [18]:
# # Align the files
# ij_stack2  = ij.py.to_java(np.array(da_stack_align,dtype='uint16'))
# ij.ui().show('PSF', ij_stack2)

### CROP PSF TO ANALYZE


In [9]:
#Go to napari and make a square for alignment
crop_region =viewer.layers['Shapes'].data[0]
template = calibration.create_box_ndarray(crop_region)
template_box = template[0:4,2:4]
print(crop_region)
print(template_box)
print(template_box.shape)

KeyError: "'Shapes' is not in list"

In [13]:
# Since OpenCV Match Template takes in uint8 || float32. We can change from uint16 to uint8 easily
# np_mip_8bit =visual.im_bit_convert(np_mip,bit=8)
min_val, max_val = template_box
t,c = template[0,0:2]
print(t,c)
crop_img = da_stack_align[:,:, min_val[0]:max_val[0],min_val[1]:max_val[1]]
# crop_img = np_mip_8bit[t,c, min_val[0]:max_val[0],min_val[1]:max_val[1]]
viewer.add_image(crop_img,name='crop')

240 8


<Image layer 'crop' at 0x1f92f811b50>

### Image J Part

In [10]:
def start_imagej():
    import imagej
    global ij
    ij = imagej.init( mode='interactive')
    # path = r'C:\Users\yoshi\Documents\fiji-win64\Fiji.app'
    # ij = imagej.init(path,mode='interactive')
    ij.ui().showUI()
    print(ij.getVersion())

QtCore.QTimer.singleShot(0, start_imagej)

In [15]:
# Align the files
ij_stack2  = ij.py.to_java(np.array(da_stack_align))
ij.ui().show('PSF', ij_stack2)

In [16]:
ij.WindowManager.getActiveWindow()
a =ij.WindowManager.getCurrentImage()
anp = ij.py.from_java(a)
print(anp.shape)
# anp2 = anp[:,:,:,np.newaxis]
# print(anp2.shape)
# anp2 = np.swapaxes(anp,1,4)
# anp2.shape

(60, 342, 413, 25)


In [17]:
anp2 = np.swapaxes(anp,1,3)
print(anp2.shape)
anp2_da = da.asarray(anp2)
viewer.add_image(anp2_da, name='crop',scale=[z_scale,1,1],multiscale=False)

(60, 25, 413, 342)


<Image layer 'crop' at 0x22682c33220>

#### Save TIFF FIles

In [21]:
import tifffile

filename = 'crop_section' + '.ome.tif'
filepath_save = os.path.join(main_folder,filename)
print(filepath_save)
tifffile.imsave(filepath_save,anp2_da)

H:\Test\20220503_M25_moreCelegans10pm\crop_section.ome.tif


C:\Users\yoshi\AppData\Local\Temp\ipykernel_17792\34166168.py:6: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(filepath_save,anp2_da)


### Read TIFF FILE

In [5]:
viewer = napari.Viewer()

In [3]:
### For preexisting tiff saved datasets
filename = 'aligned_vol.tif'
filepath_save = os.path.join(main_folder,filename)

In [5]:
filename = 'crop_section' + '.ome.tif'
filepath_save = os.path.join(main_folder,filename)

In [6]:
crop_tiff = tifffile.imread(filepath_save)
print(crop_tiff.shape)
print(type(crop_tiff))
da_crop_tiff = da.from_array(crop_tiff)

viewer.add_image(crop_tiff, name='crop',scale=[z_scale,1,1],contrast_limits=(0,2**16-1))

(600, 25, 608, 808)
<class 'numpy.ndarray'>


<Image layer 'crop' at 0x133b43873d0>

In [10]:
da_crop_tiff.shape[:,]

(600, 25, 608, 808)

c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


: 

In [9]:
# Align the files
ij_stack2  = ij.py.to_java(np.array(crop_tiff))
ij.ui().show('PSF', ij_stack2)

In [ ]:
ij.WindowManager.getActiveWindow()
a =ij.WindowManager.getCurrentImage()
anp = ij.py.from_java(a)
print(anp.shape)

In [8]:
from napari_animation import Animation
animation = Animation(viewer)
animation.animate('test.gif',canvas_only = False )

### ZARR IMPORT

In [10]:
import zarr

In [12]:
main_folder = r'H:\Test\20220504_M25_celegansOH_late1'
file_name = main_folder + '/'+'deconvolved_worm' + '.zarr'
zarr_stack = zarr.load(file_name)
da_stack_deconvolved = da.from_array(zarr_stack)


In [14]:
viewer.add_image(da_stack_deconvolved, scale= [z_scale,1,1])

<Image layer 'da_stack_deconvolved [1]' at 0x1351614dbe0>

c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_struct__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_interface__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private

### Playing with napari-animation plugin


In [41]:
from napari_animation import AnimationWidget

animation_widget = AnimationWidget(viewer)
viewer.window.add_dock_widget(animation_widget, area='right')

ModuleNotFoundError: No module named 'napari_animation'